# Air Flights
This notebook uses [OpenSky](https://opensky-network.org/)'s HTTP  Rest API to detect flights positions. 
Data is issued from a network of land probes so it will not display datas for planes currently flying over the 
ocean or uninhabited area.


In [ ]:
import os
import requests
import json
import urllib3
import pandas as pd

from jupytab_utils import Tables, DataFrameTable

# Proxy configuration
A proxy may be defined as an environment variable (HTTP_PROXY, HTTPS_PROXY) and used to retrieve datas from OpenSky Rest API

In [ ]:
http_proxy  = os.environ.get('HTTP_PROXY')
https_proxy = os.environ.get('HTTPS_PROXY')

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
proxies = {}

if http_proxy:
    proxies["http"] = http_proxy
    print("HTTP proxy defined as {}".format(http_proxy))
else:
    print("No HTTP proxy defined")
    
if https_proxy:
    proxies["https"] = https_proxy
    print("HTTPS proxy defined as {}".format(http_proxy))
else:
    print("No HTTPS proxy defined")

# OpenSky Rest API call and DataFrame creation


In [ ]:


def compute_flights():
    try:
        states = requests.get(url='https://opensky-network.org/api/states/all',
                              proxies=proxies,
                              verify=False,
                              timeout=5)
        states_json = states.json()['states']
    except Exception as e:
        states_json = [{'icao24': str(e)}]

    return pd.DataFrame(
            data=states_json,
            columns=['icao24', 'callsign', 'origin_country', 'time_position', 'last_contact',
                     'longitude', 'latitude', 'baro_altitude', 'on_ground', 'velocity',
                     'true_track', 'vertical_rate', 'sensors', 'geo_altitude', 'squawk',
                     'spi', 'position_source'])\
        .set_index('icao24')

flight_df = compute_flights()
flight_df.head(5)

## Table definition
This cell will create a Table repository that will allow to expose data through jupyter kernel gateway

In [ ]:
tables = Tables()
tables['flights'] = DataFrameTable('All flights', dataframe=flight_df, refresh_method=compute_flights)

## Schema rendering

This cell serves schema in json format. The following content is expected when running in a Jupyter notebook

`[{"id": "flights", "alias": "All flights", "columns": [{"id": "callsign", "dataType": "string"}, {"id": "origin_country", "dataType": "string"}, {"id": "time_position", "dataType": "float"}, {"id": "last_contact", "dataType": "int"}, {"id": "longitude", "dataType": "float"}, {"id": "latitude", "dataType": "float"}, {"id": "baro_altitude", "dataType": "float"}, {"id": "on_ground", "dataType":,...]`

The `# GET /schema` **MUST** be provided as the first line of the cell

In [ ]:
# GET /schema
tables.render_schema()

## Data rendering
This cell serves dataframe in json format.

The following error is expected when running this cell in a jupyter environment as Jupyter Kernel Gateway will provide the REQUEST object.
` --------------------------------------------------------------------------
 NameError                                 Traceback (most recent call last)
 <ipython-input-9-90cb191191f7> in <module>()
       1  GET /data
 ----> 2  tables.render_data(REQUEST)
 NameError: name 'REQUEST' is not defined
`

The `# GET /data` **MUST** be provided as the first line of the cell

In [ ]:
# GET /data
tables.render_data(REQUEST)